In [6]:
## instalacion de dependencias
import numpy as np
import matplotlib as mpl
import pandas as pd

In [7]:
route = "content/content_soccer/raw_data/"
players = pd.read_json(route+"players.json")
events_england = pd.read_json(route+"events/events_England.json")
# INFO General
datos = {"Players": players}

for nombre, dato in datos.items():
    print("Informacion sobre: " + nombre)
    dato.info()
    print("_"*50)
    print(" ")
    dato.isnull().sum()
    print("_"*50)
    print(" ")

Informacion sobre: Players
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3603 entries, 0 to 3602
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   passportArea           3603 non-null   object
 1   weight                 3603 non-null   int64 
 2   firstName              3603 non-null   object
 3   middleName             3603 non-null   object
 4   lastName               3603 non-null   object
 5   currentTeamId          3512 non-null   object
 6   birthDate              3603 non-null   object
 7   height                 3603 non-null   int64 
 8   role                   3603 non-null   object
 9   birthArea              3603 non-null   object
 10  wyId                   3603 non-null   int64 
 11  foot                   3603 non-null   object
 12  shortName              3603 non-null   object
 13  currentNationalTeamId  3603 non-null   object
dtypes: int64(3), object(11)
memory usage: 394.2+ 

In [15]:
pd.json_normalize(players)
players.head()

,passportArea,weight,firstName,middleName,lastName,currentTeamId,birthDate,height,role,birthArea,wyId,foot,shortName,currentNationalTeamId
0,"{'name': 'Turkey', 'id': '792', 'alpha3code': ...",78,Harun,,Tekin,4502,1989-06-17,187,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'Turkey', 'id': '792', 'alpha3code': ...",32777,right,H. Tekin,4687
1,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",73,Malang,,Sarr,3775,1999-01-23,182,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393228,left,M. Sarr,4423
2,"{'name': 'France', 'id': '250', 'alpha3code': ...",72,Over,,Mandanda,3772,1998-10-26,176,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'France', 'id': '250', 'alpha3code': ...",393230,,O. Mandanda,null
3,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",82,Alfred John Momar,,N'Diaye,683,1990-03-06,187,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'France', 'id': '250', 'alpha3code': ...",32793,right,A. N'Diaye,19314
4,"{'name': 'France', 'id': '250', 'alpha3code': ...",84,Ibrahima,,Konat\u00e9,2975,1999-05-25,192,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393247,right,I. Konat\u00e9,null


### Role

In [13]:
role = players["role"].drop_duplicates()
role = pd.json_normalize(role)
role

,code2,code3,name
0,GK,GKP,Goalkeeper
1,DF,DEF,Defender
2,MD,MID,Midfielder
3,FW,FWD,Forward


In [12]:
# Crear una lista
role_list = list(role.itertuples(index=False, name=None))
# Crear el script subEvents
with open("SQL/player/role.sql", "w") as archivo:
    archivo.write("INSERT INTO role(code2, code3, name) VALUES\n")

    valores = ",\n".join(f"('{code2}', '{code3}', '{name}')" for code2, code3, name in role_list)
    archivo.write(valores + ";\n")

### Area Passport

#### Area

In [8]:
passportArea = players["passportArea"]
passportArea = pd.json_normalize(passportArea)
passportArea = passportArea[["alpha2code","alpha3code","name"]].drop_duplicates().dropna()
area= passportArea[(passportArea["alpha2code"].notna()) & (passportArea["alpha2code"] != "") &
                      (passportArea["alpha3code"].notna()) & (passportArea["alpha3code"] != "") &
                      (passportArea["name"].notna()) & (passportArea["name"] != "")]
area

,alpha2code,alpha3code,name
0,TR,TUR,Turkey
1,SN,SEN,Senegal
2,FR,FRA,France
5,NL,NLD,Netherlands
6,BE,BEL,Belgium
...,...,...,...
3065,ZM,ZMB,Zambia
3098,BB,BRB,Barbados
3142,MS,MSR,Montserrat
3167,ZW,ZWE,Zimbabwe


In [11]:
# Crear una lista
area_list = list(area.itertuples(index=False, name=None))
# Crear el script subEvents
with open("SQL/player/area.sql", "w") as archivo:
    archivo.write("INSERT INTO area(alpha2code, alpha3code, name) VALUES\n")

    valores = ",\n".join(f"('{code2}', '{code3}', '{bytes(name.replace("'", ""), "utf-8").decode("unicode_escape")}')" for code2, code3, name in area_list)
    archivo.write(valores + ";\n")

#### passport Area

In [ ]:
players_england = events_england["playerId"].unique()
playerId = players["wyId"]
players = players[players["wyId"].isin(players_england)]
players = pd.json_normalize(players)
print(players_england)
passportArea = players_england["passportArea"]
passportArea = pd.json_normalize(passportArea)
passportArea = passportArea[["id","alpha2code"]].dropna()
#passportArea


KeyError: 'wyId'

In [47]:
players_england = events_england["playerId"].unique()
len(players_england)

515

In [48]:
resultado2 = players[players["wyId"].isin(players_england)]
len(resultado2)

514

In [51]:
resultado = players[players["currentTeamId"].isin(events_england["teamId"])]
len(resultado["currentTeamId"].unique())

20

In [59]:
#pd.json_normalize(players["birthArea"])
#players[players["birthArea"]  == "England"]


diferencia = set(players_england) - set(resultado2["wyId"])
#int(diferencia.pop())
diferencia
#print(set(resultado2["wyId"]).intersection(set(players_england)))



{np.int64(0)}

In [64]:
events_england[events_england["playerId"]==0]
#players[players["wyId"] == 0]

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
8,1,Air duel,"[{'id': 703}, {'id': 1801}]",0,"[{'y': 85, 'x': 61}, {'y': 80, 'x': 67}]",2499719,Duel,1609,1H,14.765321,10,177961218
23,1,Ground loose ball duel,"[{'id': 701}, {'id': 1802}]",0,"[{'y': 78, 'x': 33}, {'y': 85, 'x': 29}]",2499719,Duel,1609,1H,38.191874,13,177961219
36,3,Free Kick,[{'id': 1801}],0,"[{'y': 44, 'x': 65}, {'y': 31, 'x': 65}]",2499719,Free Kick,1609,1H,74.737141,31,177959198
41,1,Ground defending duel,"[{'id': 502}, {'id': 701}, {'id': 1802}]",0,"[{'y': 86, 'x': 4}, {'y': 97, 'x': 10}]",2499719,Duel,1631,1H,85.074043,12,177961220
54,1,Ground attacking duel,"[{'id': 701}, {'id': 1802}]",0,"[{'y': 27, 'x': 89}, {'y': 27, 'x': 95}]",2499719,Duel,1631,1H,152.127886,11,177961221
...,...,...,...,...,...,...,...,...,...,...,...,...
643080,5,Ball out of the field,[],0,"[{'y': 100, 'x': 51}, {'y': 100, 'x': 100}]",2500098,Interruption,1623,2H,2589.146520,50,251596374
643101,3,Free Kick,[{'id': 1801}],0,"[{'y': 87, 'x': 40}, {'y': 80, 'x': 56}]",2500098,Free Kick,1623,2H,2665.580593,31,251596391
643117,1,Ground defending duel,"[{'id': 502}, {'id': 701}, {'id': 1802}]",0,"[{'y': 34, 'x': 7}, {'y': 28, 'x': 5}]",2500098,Duel,1623,2H,2712.941293,12,251596987
643140,3,Free Kick,[{'id': 1801}],0,"[{'y': 81, 'x': 70}, {'y': 70, 'x': 59}]",2500098,Free Kick,1633,2H,2780.300522,31,251596224
